In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

# Prepare dataset

In [2]:
# Load all gbtc data
df_minute_gbtc_2020_12_01_2021_04_18 = pd.read_csv("../data/raw/gbtc_minute_2020-12-01_2021-04-18.csv", sep=",", index_col='date')
df_minute_gbtc_2021_04_18_2021_05_01 = pd.read_csv("../data/raw/gbtc_minute_2021-04-18_2021-05-01.csv", sep=",", index_col='date')
df_minute_gbtc_2020_12_01_2021_04_18.index.astype('datetime64[ns]')
df_minute_gbtc_2021_04_18_2021_05_01.index.astype('datetime64[ns]')

df_gbtc_minute = pd.concat([df_minute_gbtc_2020_12_01_2021_04_18, df_minute_gbtc_2021_04_18_2021_05_01]).drop_duplicates().sort_index(ascending=True)
df_gbtc_minute['time'] = (df_gbtc_minute['datetime'] / 1000).astype('int64')
# df_min = pd.concat([df_minute_gbtc_2020_12_01_2021_04_18, {more data in future}]).drop_duplicates().sort_index(ascending=True)

# Load all btc data
df_minute_btc_usd_minute_2020_06_01_2021_04_18 = pd.read_csv("../data/raw/btc-usd_minute_2020-06-01_2021-04-18.csv", sep=",", index_col='date')
df_minute_btc_usd_minute_2020_06_01_2021_04_18.index.astype('datetime64[ns]')
df_minute_btc_usd_minute_2021_04_18_2021_05_01 = pd.read_csv("../data/raw/btc-usd_minute_2021-04-18_2021-05-01.csv", sep=",", index_col='date')
df_minute_btc_usd_minute_2021_04_18_2021_05_01.index.astype('datetime64[ns]')

df_btc_minute = pd.concat([df_minute_btc_usd_minute_2020_06_01_2021_04_18, df_minute_btc_usd_minute_2021_04_18_2021_05_01]).drop_duplicates().sort_index(ascending=True)

df_btc_minute = df_btc_minute.sort_index(ascending=True)
df_btc_minute = df_btc_minute['2021-03-01': ]



In [3]:
# Join btc with gbtc on time.
df_minute = pd.merge(df_btc_minute, df_gbtc_minute, on="time", how="left")

df_minute = df_minute.rename(columns = {'low_price':'btc_low_price', 'high_price':'btc_high_price', 'close_price':'btc_close_price', 'open_price':'btc_open_price', 'volume_x':'btc_volume', 'time':'time', 'open':'gbtc_open_price', 'high':'gbtc_high_price', 
'low':'gbtc_low_price', 'close':'gbtc_close_price', 'volume_y':'gbtc_volume'})

df_minute = df_minute.drop(columns=['datetime'])
df_minute['datetime'] = df_minute['time'].apply(lambda d: datetime.utcfromtimestamp(d)) 
df_minute = df_minute.set_index('datetime')
df_minute.index.astype('datetime64[ns]')

btc_per_share = (0.00094509 + 0.000944257) / 2 

df_minute["nav_open_price"] = df_minute["btc_open_price"] * btc_per_share
df_minute["nav_close_price"] = df_minute["btc_close_price"] * btc_per_share
df_minute["nav_high_price"] = df_minute["btc_high_price"] * btc_per_share
df_minute["nav_low_price"] = df_minute["btc_low_price"] * btc_per_share

df_minute["premium"] = (df_minute["gbtc_close_price"] - df_minute["nav_close_price"]) / df_minute["nav_close_price"]
df_minute["premium_high"] = (df_minute["gbtc_high_price"] - df_minute["nav_high_price"]) / df_minute["nav_high_price"]
df_minute["premium_open"] = (df_minute["gbtc_open_price"] - df_minute["nav_open_price"]) / df_minute["nav_open_price"]
df_minute["premium_close"] = (df_minute["gbtc_close_price"] - df_minute["nav_close_price"]) / df_minute["nav_close_price"]
df_minute["premium_low"] = (df_minute["gbtc_low_price"] - df_minute["nav_low_price"]) / df_minute["nav_low_price"]
# df_minute = df_minute.dropna()
df_minute.to_csv("../data/btc_gbtc/btc_gbtc_minute_2021.csv")

In [4]:
# Add extra information
from ta.trend import SMAIndicator, EMAIndicator, ADXIndicator, MACD
from ta.momentum import RSIIndicator

# 1) preimum Simple moving average 
df_minute_pure_gbtc = df_minute.drop_duplicates().dropna()
indicator_sma_15 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=15, fillna=True)
premium_sma15 = indicator_sma_15.sma_indicator().dropna().to_frame()
df_indicators = df_minute.join(premium_sma15)

indicator_sma_30 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=30, fillna=True)
premium_sma30 = indicator_sma_30.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma30)

indicator_sma_60 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=60, fillna=True)
premium_sma60 = indicator_sma_60.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma60)

indicator_sma_120 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=120, fillna=True)
premium_sma120 = indicator_sma_120.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma120)

indicator_sma_180 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=180, fillna=True)
premium_sma180 = indicator_sma_180.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma180)

# 2) Premium Exponential moving average
indicator_ema_15 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=15, fillna=True)
premium_ema15 = indicator_ema_15.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema15)

indicator_ema_30 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=30, fillna=True)
premium_ema30 = indicator_ema_30.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema30)

indicator_ema_60 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=60, fillna=True)
premium_ema60 = indicator_ema_60.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema60)

indicator_ema_120 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=120, fillna=True)
premium_ema120 = indicator_ema_120.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema120)

indicator_ema_180 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=180, fillna=True)
premium_ema180 = indicator_ema_180.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema180)

# 3) ADX (premium, btc)
premium_adx_indicator = ADXIndicator(high=df_minute_pure_gbtc["premium_high"], low=df_minute_pure_gbtc["premium_low"], close=df_minute_pure_gbtc["premium_close"], window=14, fillna=False)
premium_adx = premium_adx_indicator.adx().dropna().to_frame() 
premium_adx = premium_adx.rename(columns={'adx':'premium_adx'})
premium_adx_neg = premium_adx_indicator.adx_neg().dropna().to_frame() 
premium_adx_neg = premium_adx_neg.rename(columns={'adx_neg':'premium_adx_neg'})
premium_adx_pos = premium_adx_indicator.adx_pos().dropna().to_frame()
premium_adx_pos = premium_adx_pos.rename(columns={'adx_pos':'premium_adx_pos'})

# btc_adx_indicator = ADXIndicator(high=df_minute["btc_high_price"], low=df_minute["btc_low_price"], close=df_minute["btc_close_price"], window=14, fillna=True)
df_indicators = df_indicators.join(premium_adx)
df_indicators = df_indicators.join(premium_adx_neg)
df_indicators = df_indicators.join(premium_adx_pos)

btc_adx_indicator = ADXIndicator(high=df_minute["btc_open_price"], low=df_minute["btc_low_price"], close=df_minute["btc_close_price"], window=14, fillna=True)
btc_adx = btc_adx_indicator.adx().dropna().to_frame() 
btc_adx = btc_adx.rename(columns={'adx':'btc_adx'})
btc_adx_neg = btc_adx_indicator.adx_neg().dropna().to_frame() 
btc_adx_neg = btc_adx_neg.rename(columns={'adx_neg':'btc_adx_neg'})
btc_adx_pos = btc_adx_indicator.adx_pos().dropna().to_frame() 
btc_adx_pos = btc_adx_pos.rename(columns={'adx_pos':'btc_adx_pos'})

df_indicators = df_indicators.join(btc_adx)
df_indicators = df_indicators.join(btc_adx_neg)
df_indicators = df_indicators.join(btc_adx_pos)

# 4) MACD (premium, btc)
premium_macd_indicator = MACD(close=df_minute_pure_gbtc["premium_close"], window_slow=26, window_fast=12, window_sign=9, fillna=True)
premium_macd_line = premium_macd_indicator.macd().dropna().to_frame() 
premium_macd_line = premium_macd_line.rename(columns={'MACD_12_26':'premium_macd_12_26'})
premium_macd_histo = premium_macd_indicator.macd_diff().dropna().to_frame() 
premium_macd_histo = premium_macd_histo.rename(columns={'MACD_diff_12_26':'premium_macd_histo_12_26'})
premium_macd_signal = premium_macd_indicator.macd_signal().dropna().to_frame() 
premium_macd_signal = premium_macd_signal.rename(columns={'MACD_sign_12_26':'premium_macd_sign_12_26'})

df_indicators = df_indicators.join(premium_macd_line)
df_indicators = df_indicators.join(premium_macd_histo)
df_indicators = df_indicators.join(premium_macd_signal)

btc_macd_indicator = MACD(close=df_minute["btc_close_price"], window_slow=26, window_fast=12, window_sign=9, fillna=True)
btc_macd_line = btc_macd_indicator.macd().dropna().to_frame() 
btc_macd_line = btc_macd_line.rename(columns={'MACD_12_26':'btc_macd_12_26'})
btc_macd_histo = btc_macd_indicator.macd_diff().dropna().to_frame() 
btc_macd_histo = btc_macd_histo.rename(columns={'MACD_diff_12_26':'btc_macd_histo_12_26'})
btc_macd_signal = btc_macd_indicator.macd_signal().dropna().to_frame() 
btc_macd_signal = btc_macd_signal.rename(columns={'MACD_sign_12_26':'btc_macd_sign_12_26'})

df_indicators = df_indicators.join(btc_macd_line)
df_indicators = df_indicators.join(btc_macd_histo)
df_indicators = df_indicators.join(btc_macd_signal)

# 5) SCI (btc)
btc_rsi_indicator = RSIIndicator(close=df_minute["btc_close_price"], window = 14, fillna = True)
btc_rsi = btc_rsi_indicator.rsi()
df_indicators = df_indicators.join(btc_rsi)

df_indicators.to_csv("../data/btc_gbtc/btc_gbtc_minute_with_indicators_2021.csv")


/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [5]:
btc_rsi[:30]

datetime
2021-03-01 00:00:00    100.000000
2021-03-01 00:01:00    100.000000
2021-03-01 00:02:00     29.747539
2021-03-01 00:03:00     22.538189
2021-03-01 00:04:00     22.018291
2021-03-01 00:05:00     17.865514
2021-03-01 00:06:00     21.017857
2021-03-01 00:07:00     29.772702
2021-03-01 00:08:00     53.662521
2021-03-01 00:09:00     56.599731
2021-03-01 00:10:00     50.714442
2021-03-01 00:11:00     56.583643
2021-03-01 00:12:00     57.249009
2021-03-01 00:13:00     55.705189
2021-03-01 00:14:00     68.002263
2021-03-01 00:15:00     73.168161
2021-03-01 00:16:00     80.112499
2021-03-01 00:17:00     84.959940
2021-03-01 00:18:00     85.267569
2021-03-01 00:19:00     84.065001
2021-03-01 00:20:00     84.643635
2021-03-01 00:21:00     77.144072
2021-03-01 00:22:00     77.642639
2021-03-01 00:23:00     70.862367
2021-03-01 00:24:00     73.822892
2021-03-01 00:25:00     72.923700
2021-03-01 00:26:00     73.233579
2021-03-01 00:27:00     73.016631
2021-03-01 00:28:00     71.496011
2021-

In [6]:
import datetime
# theday = datetime.date.today()
# weekday = theday.isoweekday()
# # The start of the week
# start = theday - datetime.timedelta(days=weekday)
# # build a simple range
# dates = [start + datetime.timedelta(days=d) for d in range(7)]

# # Starts with knowing the day of the week
# week_day=datetime.datetime.now().isocalendar()[2]

# # Calculates Starting date (Sunday) for this case by subtracting current date with time delta of the day of the week
# start_date=datetime.datetime.now() - datetime.timedelta(days=week_day)

# # Prints the list of dates in a current week
# dates=[str((start_date + datetime.timedelta(days=i)).date()) for i in range(7)]
# dates

In [18]:
# divide data to weeks
weeks = [ ("2021-03-03", "2021-03-08"), ("2021-03-08", "2021-03-15"), ("2021-03-15", "2021-03-22"), ("2021-03-22", "2021-03-29"), ("2021-03-29", "2021-04-04"), ("2021-04-04", "2021-04-12"), ("2021-04-12", "2021-04-19")]

df_indicators.index[0]
start_date = weeks[0][0]
end_date = weeks[0][1]
mask = (df_indicators.index > start_date) & (df_indicators.index <= end_date)
df = df_indicators.loc[mask]

df

,btc_low_price,btc_high_price,btc_close_price,btc_open_price,btc_volume,time,gbtc_open_price,gbtc_high_price,gbtc_low_price,gbtc_close_price,...,btc_adx,btc_adx_neg,btc_adx_pos,premium_macd_12_26,premium_macd_histo_12_26,premium_macd_sign_12_26,btc_macd_12_26,btc_macd_histo_12_26,btc_macd_sign_12_26,rsi
datetime,,,,,,,,,,,,,,,,,,,,,
2021-03-03 00:01:00,48350.00,48447.52,48400.00,48366.63,8.610439,1614729660,NaN,NaN,NaN,NaN,...,21.050048,31.460001,68.349770,NaN,NaN,NaN,35.125344,-2.630171,37.755514,55.390097
2021-03-03 00:02:00,48364.12,48453.34,48393.06,48400.01,7.615530,1614729720,NaN,NaN,NaN,NaN,...,22.501203,29.112279,70.189786,NaN,NaN,NaN,34.071102,-2.947530,37.018632,54.755095
2021-03-03 00:03:00,48300.35,48393.29,48339.23,48393.22,6.558895,1614729780,NaN,NaN,NaN,NaN,...,22.579227,35.921679,58.106174,NaN,NaN,NaN,28.562724,-6.764727,35.327450,49.969886
2021-03-03 00:04:00,48300.84,48349.38,48302.75,48335.42,8.068158,1614729840,NaN,NaN,NaN,NaN,...,22.651677,33.364644,53.969966,NaN,NaN,NaN,21.011456,-11.452796,32.464251,46.973846
2021-03-03 00:05:00,48161.75,48305.69,48161.75,48305.69,25.558776,1614729900,NaN,NaN,NaN,NaN,...,21.478555,47.489209,41.920785,NaN,NaN,NaN,3.607910,-23.085073,26.692983,37.592080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-07 23:56:00,51250.00,51389.48,51268.08,51385.38,24.112690,1615161360,NaN,NaN,NaN,NaN,...,51.694628,38.281050,71.242906,NaN,NaN,NaN,104.088748,7.306930,96.781818,58.166297
2021-03-07 23:57:00,51140.36,51266.67,51156.41,51266.67,18.833238,1615161420,NaN,NaN,NaN,NaN,...,48.564474,48.180555,56.414789,NaN,NaN,NaN,86.043658,-8.590529,94.634186,48.605041
2021-03-07 23:58:00,51100.00,51174.78,51108.34,51154.02,20.301886,1615161480,NaN,NaN,NaN,NaN,...,45.169210,50.285463,51.332933,NaN,NaN,NaN,67.090568,-22.034895,89.125463,45.163496


# Graph

In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class IndicatorType:
    ADX_PREMIUM = 1
    ADX_BTC = 2
    MACD_PREMIUM = 3
    MACD_BTC = 4
    RSI = 5

def plot_price_with_premium_chart(df, chart_width=5000, chart_height=800, nticks=200, ema=False, indicator_type=IndicatorType.ADX_PREMIUM):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df.index, y=df["nav_close_price"], mode='lines+markers', name="btc nav close price"), secondary_y=False)
    fig.add_trace(go.Scatter(x=df.index, y=df["gbtc_close_price"], mode='lines+markers', name="gbtc close price"), secondary_y=False)
    fig.add_trace(go.Scatter(x=df.index, y=df["premium"], mode='lines+markers', name="premium percentage"), secondary_y=True)
    
    if ema:
        fig.add_trace(go.Scatter(x=df.index, y=df["ema_15"], mode='lines+markers', name="premium 15 EMA"), secondary_y=True)
        # fig.add_trace(go.Scatter(x=df.index, y=df["ema_30"], mode='lines+markers', name="premium 30 EMA"), secondary_y=True)
        # fig.add_trace(go.Scatter(x=df.index, y=df["ema_60"], mode='lines+markers', name="premium 60 EMA"), secondary_y=True)
        # fig.add_trace(go.Scatter(x=df.index, y=df["ema_120"], mode='lines+markers', name="premium 120 EMA"), secondary_y=True)
        fig.add_trace(go.Scatter(x=df.index, y=df["ema_180"], mode='lines+markers', name="premium 180 EMA"), secondary_y=True)
    else:
        # fig.add_trace(go.Scatter(x=df.index, y=df["sma_15"], mode='lines+markers', name="premium 15 SMA"), secondary_y=True)
        # fig.add_trace(go.Scatter(x=df.index, y=df["sma_30"], mode='lines+markers', name="premium 30 SMA"), secondary_y=True)
        # fig.add_trace(go.Scatter(x=df.index, y=df["sma_60"], mode='lines+markers', name="premium 60 SMA"), secondary_y=True)
        # fig.add_trace(go.Scatter(x=df.index, y=df["sma_120"], mode='lines+markers', name="premium 120 SMA"), secondary_y=True)
        fig.add_trace(go.Scatter(x=df.index, y=df["sma_180"], mode='lines+markers', name="premium 180 SMA"), secondary_y=True)

    if indicator_type == IndicatorType.ADX_PREMIUM:
        fig.add_trace(go.Scatter(x=df.index, y=df["premium_adx"], mode='lines+markers', name="premium adx"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["premium_adx_neg"], mode='lines+markers', name="premium adx -DI"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["premium_adx_pos"], mode='lines+markers', name="premium adx +DI"), secondary_y=False)
    elif indicator_type == IndicatorType.ADX_BTC:
        fig.add_trace(go.Scatter(x=df.index, y=df["btc_adx"], mode='lines+markers', name="btc adx"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["btc_adx_neg"], mode='lines+markers', name="btc adx -DI"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["btc_adx_pos"], mode='lines+markers', name="btc adx +DI"), secondary_y=False)
    elif indicator_type == IndicatorType.RSI:
        fig.add_trace(go.Scatter(x=df.index, y=df["rsi"], mode='lines+markers', name="btc rsi"), secondary_y=False)
    elif indicator_type == IndicatorType.MACD_BTC:
        fig.add_trace(go.Scatter(x=df.index, y=df["btc_macd_12_26"], mode='lines+markers', name="btc macd"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["btc_macd_sign_12_26"], mode='lines+markers', name="btc macd sign"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["btc_macd_histo_12_26"].apply(lambda v: abs(v)), mode='lines+markers', name="btc macd histo"), secondary_y=False)
    elif indicator_type == IndicatorType.MACD_PREMIUM:
        fig.add_trace(go.Scatter(x=df.index, y=df["premium_macd_12_26"], mode='lines+markers', name="premium macd"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["premium_macd_sign_12_26"], mode='lines+markers', name="premium macd sign"), secondary_y=False)
        fig.add_trace(go.Scatter(x=df.index, y=df["premium_macd_histo_12_26"].apply(lambda v: abs(v)), mode='lines+markers', name="premium macd histo"), secondary_y=False)
    else:
        print("Nothing ...")

    fig.update_layout(
        autosize=False,
        width=chart_width,
        height=chart_height,
        paper_bgcolor="LightSteelBlue",
    )

    fig.update_yaxes(title_text="price", secondary_y=False)
    fig.update_yaxes(title_text="premium to gbtc price percentage", secondary_y=True)

    fig.update_xaxes(nticks=nticks)          
    fig.show()

In [23]:
# total 7 weeks since Mar 3, try increase the week_no for other week's data
week_no = 1
start_date = weeks[week_no][0]34
end_date = weeks[week_no][1]
mask = (df_indicators.index > start_date) & (df_indicators.index <= end_date)
df = df_indicators.loc[mask]

# plot premium and its average 
plot_price_with_premium_chart(df, chart_width=5000, indicator_type=IndicatorType.MACD_BTC)
# plot_price_with_premium_chart(df, chart_width=5000, ema = True)